In [ ]:
!pip install openai
!pip install langchain

In [3]:
from os.path import join as pjoin
import pandas as pd

# Step 1: Loading file with cleaned hadiths. This file contains about 34k hadiths

In [4]:
hadith_file = 'datasets/cleaned_hadiths.csv'
hadith_df = pd.read_csv(pjoin(hadith_file))

# Step 2: Creating a question such that the hadith is the answer to a given question using LLMs. This step is needed for fine-tuning chat completion models

In [5]:
import openai
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin
import textwrap
import ast
import pandas as pd
import datetime

# Set the API key for OpenAI
openai.api_key = 'YOUR KEY HERE'


def get_prompt_for_hadith(sample_hadith):
  prompt = f"The following text is a hadith {sample_hadith}. I am trying to train a language model that requires the text data to be fed as a prompt and a completion. Can you write a question such that the hadith I shared will be seen as an appropriate completion? If the question has a pronoun or refers to the Prophet Muhammad (saw), make sure to add (saw) or (peace be upon him) next to the pronouns referring to the prophet Muhammad (peace be upon him)"
  try:
    # Get the model's response
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=200
    )
    # Process the model's response and update the schema
    # This assumes that the model's response is a dictionary that can be used to update your schema
    # The exact method you use to process the response will depend on how the model responds
    result = response.choices[0].text.strip()
    return result
  except:
      print('Error')
      return None



# Step 3: Running hadiths against propmpt engine one by one. Can be expensive so also an option to do it on a subset. We provide a file with 10k hadiths

In [ ]:
N = 10000
#N = len(hadith_df)
first_N_hadith_df = hadith_df[:N]

In [ ]:
prompt_for_hadiths = []
error_message = 'N/A'
print('Start time for ',str(N),'requests is: ',str(datetime.datetime.now()))
for counter in range(len(first_N_hadith_df)):
  if counter %50 == 0:
    print(counter)
    print(datetime.datetime.now())
  sample_hadith = first_N_hadith_df[counter]
  prompt = get_prompt_for_hadith(sample_hadith)
  if prompt:
    prompt_for_hadiths.append(prompt)
  else:
    prompt_for_hadiths.append(error_message)
print('End time for ',str(N),'requests is: ',str(datetime.datetime.now()))


In [ ]:
file_name = str(N)+'_hadiths_with_prompts.csv'
first_N_hadith_df.to_csv(file_name)